from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)

# Install Dependencies

In [0]:
!pip install docx2txt
!pip install PyMuPDF
!pip install spacy==2.0.16
!pip install msgpack==0.5.6

# Data in json format downloaded from dataturks website


In [0]:
TRAIN_DIR = "/content/gdrive/My Drive/resume_1.json"

In [0]:
import json
import random
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score
import spacy


In [0]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        lines = [json.loads(line) for line in open(dataturks_JSON_FilePath, 'r',encoding='UTF-8')]
        for line in lines:
            
            data = line
            
            text = data['content']
            entities = []
            for annot in data['annotation']:
            
                
            
                point = annot['points'][0]

                labels = annot['label']
      
               
                if not isinstance(labels, list):
         
                    labels = [labels]
    

                for label in labels:

                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))



            training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

In [0]:
TRAIN_DATA = convert_dataturks_to_spacy(TRAIN_DIR)
nlp = spacy.blank('en') 
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
    

for _, annotations in TRAIN_DATA:
      for ent in annotations.get('entities'):
        ner.add_label(ent[2])


other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(30):
        print("Statring iteration " + str(itn))
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            nlp.update(
                [text],  
                [annotations],  
                drop=0.3,  
                sgd=optimizer, 
                losses=losses)
        print(losses)


In [0]:
nlp.to_disk("/content/gdrive/My Drive/my_model")
print("Saved model ")
